In [1]:
import pandas as pd
from binance.spot import Spot
from ta.utils import dropna

In [2]:
binance_client = Spot()
symbol = "SOLUSDT"
interval = '15m'
limit = '1000'

In [3]:
def get_data(binance_client: Spot, symbol: str, interval: str, limit: int) -> pd.DataFrame:
    df = pd.DataFrame(binance_client.klines(symbol=symbol, interval=interval, limit=limit))
    df = df.iloc[:, :9]
    df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time', 'Quote_av', 'Trades']

    # drop unnecessary columns
    df = df.drop(['Close_time', 'Quote_av', 'Trades'], axis=1)

    # convert to datetime
    df['Time'] = pd.to_datetime(df['Time'], unit='ms')

    # set index
    df = df.set_index('Time')
    # convert to float
    df = df.astype(float)

    return df

In [4]:
# df = get_data(binance_client, symbol, interval, limit)
# df

In [5]:
df = pd.read_csv('historical-data/15min/ADA-USD-15m-2021.csv', sep=',')
df = df.drop('Unnamed: 6', axis=1) # Drop the 'Unnamed: 6' column
df = dropna(df)
df['Time'] = pd.to_datetime(df['Time']) # Convert the 'Datetime' column to datetime format
df = df.set_index('Time') # Set the 'Datetime' column as the index
df

,Open,High,Low,Close,Volume
Time,,,,,
2021-03-18 16:00:00,1.6551,1.8500,1.4650,1.8490,13671421.86
2021-03-18 16:15:00,1.8498,1.8800,1.3600,1.4025,26576990.27
2021-03-18 16:30:00,1.4024,1.4500,1.3400,1.3500,11549407.35
2021-03-18 16:45:00,1.3489,1.3800,1.3454,1.3559,7405071.24
2021-03-18 17:00:00,1.3559,1.3841,1.3500,1.3799,4752933.63
...,...,...,...,...,...
2023-12-17 21:15:00,0.5940,0.5947,0.5904,0.5908,309338.73
2023-12-17 21:30:00,0.5907,0.5919,0.5889,0.5893,249179.77
2023-12-17 21:45:00,0.5893,0.5897,0.5873,0.5887,362002.77


In [6]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [7]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [8]:
from backtesting import Backtest

bt = Backtest(df, SmaCross, cash=10_000, commission=.002)


In [9]:
stats = bt.optimize(n1=range(5, 100, 5),
                    n2=range(10, 200, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

/home/a/Desktop/algo-trading/v11-learning-backtestingpy/venv/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 551 configurations.
  output = _optimize_grid()


Start                     2021-03-18 16:00:00
End                       2023-12-17 22:15:00
Duration                   1004 days 06:15:00
Exposure Time [%]                   99.895209
Equity Final [$]                  5941.063025
Equity Peak [$]                  18207.021353
Return [%]                          -40.58937
Buy & Hold Return [%]              -68.215251
Return (Ann.) [%]                  -17.230361
Volatility (Ann.) [%]               86.014444
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -73.628944
Avg. Drawdown [%]                   -11.90468
Max. Drawdown Duration      935 days 09:45:00
Avg. Drawdown Duration       45 days 14:15:00
# Trades                                 1134
Win Rate [%]                        34.479718
Best Trade [%]                      30.866585
Worst Trade [%]                    -11.199197
Avg. Trade [%]                    

In [10]:
stats._strategy

<Strategy SmaCross(n1=30,n2=100)>

In [11]:
bt.plot(plot_volume=False, plot_pl=False)

/home/a/Desktop/algo-trading/v11-learning-backtestingpy/venv/lib/python3.10/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


TypeError: Index.get_loc() got an unexpected keyword argument 'method'